In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics


def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x, y


(x, y), (x_test, y_test) = datasets.fashion_mnist.load_data()
print(x.shape, y.shape)     # check

batchsz = 128

db = tf.data.Dataset.from_tensor_slices((x, y))  # 訓練
db = db.map(preprocess).shuffle(10000).batch(batchsz)

db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))  # 測試
db_test = db_test.map(preprocess).batch(batchsz)

db_iter = iter(db)      # 迭代器
sample = next(db_iter)  # 看他的下個個迭代
print('batch', sample[0].shape, sample[1].shape)    # sample[0]指x sample[1]指y

model = Sequential([
    layers.Dense(256, activation=tf.nn.relu),   # [b, 784] => [b, 256]
    layers.Dense(128, activation=tf.nn.relu),   # [b, 256] => [b, 128]
    layers.Dense(64, activation=tf.nn.relu),    # [b, 128] => [b, 64]
    layers.Dense(32, activation=tf.nn.relu),    # [b, 64] => [b, 32]
    layers.Dense(10)                            # [b, 32] => [b, 10]
])

model.build(input_shape=[None, 28*28])
model.summary()
optimizers = optimizers.Adam(learning_rate=1e-3)


def main():
    for epoch in range(30):

        for step, (x, y) in enumerate(db):
            # x: [b, 28, 28] => [b, 784]
            # y: [b]
            x = tf.reshape(x, [-1, 28*28])

            with tf.GradientTape() as tape:
                # [b, 784] => [b, 10]
                logits = model(x)
                y_onehot = tf.one_hot(y, depth=10)
                # [b]
                loss_mse = tf.reduce_mean(tf.losses.MSE(y_onehot, logits))
                loss_ce = tf.reduce_mean(tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True))

            grads = tape.gradient(loss_mse, model.trainable_variables)
            optimizers.apply_gradients(zip(grads, model.trainable_variables))
            # zip(x, y)可以將x[0],y[0]拚再一起舉例  list1 = [1, 2, 3]
            # list2 = ['a', 'b', 'c']
            # zipped = zip(list1, list2)
            # print(list(zipped))  # 輸出：[(1, 'a'), (2, 'b'), (3, 'c')]
            # optimizer.apply_gradients(grads_and_vars, global_step=None,
            # name = None)
            # 其中grads_and_vars需要的即為一個由梯度和對應的參數組成的配對列表
            # 組合下來即可原地根據梯度更新參數 w = w -lr*grad
            if step % 100 == 0:
                print(epoch, step, 'loss:', float(loss_ce), float(loss_mse))
        # test
        total_correct = 0
        total_num = 0
        for x, y in db_test:
            # x: [b, 28, 28] => [b, 784]
            # y: [b]
            x = tf.reshape(x, [-1, 28*28])
            logits = model(x)
            #logits => prob [b,10]
            prob = tf.nn.softmax(logits, axis=1)
            # [b, 10] => [b]
            pred = tf.argmax(prob, axis=1)
            pred = tf.cast(pred, dtype=tf.int32)
            # pred:[b]
            # y: [b]
            correct = tf.equal(pred, y)
            correct = tf.reduce_sum(tf.cast(correct, dtype=tf.int32))
            total_correct += int(correct)
            total_num += x.shape[0]
            
        acc = total_correct / total_num
        print(epoch, 'acc:', acc)
if __name__ == '__main__':
    main()


(60000, 28, 28) (60000,)
batch (128, 28, 28) (128,)
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_51 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_52 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_53 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_54 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_55 (Dense)             (None, 10)                170       
Total params: 244,890
Trainable params: 244,890
Non-trainable param